In [ ]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import re
import requests
from tqdm import tqdm

In [2]:
# 축산계 자료 불러오기
축산계_원본 = pd.read_excel(
    "E:/Coding/TMDL/전국오염원조사/축산계/2023년기준_전국오염원_조사자료_축산계_가축분뇨현황.xlsx",
    skiprows=6, header=None
)
# 연도를 첫번째 열에 추가
축산계_원본.insert(0, "연도", 2023)

축산계_원본

,연도,0,1,2,3,4,5,6,7,8,...,85,86,87,88,89,90,91,92,93,94
0,2023,성기춘,성기춘,4181000-52-2001-00003,NaN,210001,성기춘210001,NaN,NaN,5111012100,...,NaN,NaN,NaN,NaN,밭,5.111012e+09,강원특별자치도,춘천시,온의동,NaN
1,2023,이재호,이재호,4181000-52-2017-00007,NaN,200767,이재호200767,NaN,NaN,5111012500,...,NaN,NaN,NaN,NaN,밭,5.111012e+09,강원특별자치도,춘천시,삼천동,NaN
2,2023,칠전농장,지찬림,4181000-52-2002-00011,NaN,NaN,지찬림,NaN,NaN,5111012600,...,NaN,NaN,NaN,NaN,밭,5.111013e+09,강원특별자치도,춘천시,칠전동,NaN
3,2023,칠전농장,지찬림,4181000-52-2006-00034,NaN,NaN,지찬림,NaN,NaN,5111012600,...,NaN,NaN,NaN,NaN,밭,5.111013e+09,강원특별자치도,춘천시,칠전동,NaN
4,2023,춘천승마장,춘천시장(체육청소년과),4181000-52-2016-00014,NaN,NaN,춘천시장(체육청소년과),NaN,NaN,5111012700,...,NaN,NaN,NaN,NaN,밭,5.111013e+09,강원특별자치도,춘천시,송암동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,2023,이지환,이지환,4351000-52-2018-00010,NaN,202178,NaN,NaN,NaN,5183035038,...,NaN,NaN,NaN,NaN,밭,5.183035e+09,강원특별자치도,양양군,강현면,장산리
11082,2023,털보농장,전성만,4350000-033-2022-0001,NaN,203676,NaN,NaN,NaN,5183035038,...,NaN,NaN,NaN,NaN,밭,5.183035e+09,강원특별자치도,양양군,강현면,장산리
11083,2023,홍성훈,홍성훈,4351000-52-2019-00001,NaN,202012,NaN,NaN,NaN,5183035038,...,NaN,NaN,NaN,NaN,밭,5.183035e+09,강원특별자치도,양양군,강현면,장산리
11084,2023,강산농장,오민석,NaN,NaN,203754,NaN,NaN,NaN,5183035038,...,NaN,NaN,NaN,NaN,밭,5.183035e+09,강원특별자치도,양양군,강현면,장산리


In [ ]:
# 동리_유역_현황 자료 읽기 및 전처리 
단위유역 = pd.read_excel("E:/Coding/TMDL/전국오염원조사/동리_유역_현황.xlsx")

# 동리코드 생성: 시군, 읍면동, 그리고 리가 결측이면 읍면동 사용
단위유역["동리코드"] = 단위유역.apply(
    lambda row: f"{row['시군']} {row['읍면동']} {row['리'] if pd.notnull(row['리']) and row['리'] != '' else row['읍면동']}",
    axis=1
)

# 필요한 열만 선택
#단위유역 = 단위유역[["법정동명", "단위유역", "점유율"]]

# 점유율을 소수점으로 변환
#단위유역["점유율"] = 단위유역["점유율"].astype(str).str.replace("%", "").astype(float) / 100

# 점유율이 100인 행만 선택
#단위유역 = 단위유역[단위유역["점유율"] == 100]

# 점유율 열 삭제
#단위유역 = 단위유역.drop(columns=["점유율"])

단위유역

,동리,법정동코드,법정동명,시도,시군,읍면동,리,동리면적,단위유역,유역면적,유역면적비율,점유율,동리개수,동리코드
0,봉의동,5111010100,강원특별자치도 춘천시 봉의동,강원특별자치도,춘천시,봉의동,NaN,1.774445e+05,북한C,1.774445e+05,100.0,100.0,1,춘천시 봉의동 봉의동
1,요선동,5111010200,강원특별자치도 춘천시 요선동,강원특별자치도,춘천시,요선동,NaN,8.835030e+04,북한C,8.835030e+04,100.0,100.0,1,춘천시 요선동 요선동
2,낙원동,5111010300,강원특별자치도 춘천시 낙원동,강원특별자치도,춘천시,낙원동,NaN,1.079278e+05,북한C,1.079278e+05,100.0,100.0,1,춘천시 낙원동 낙원동
3,중앙로1가,5111010400,강원특별자치도 춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,NaN,3.747000e+04,북한C,3.747000e+04,100.0,100.0,1,춘천시 중앙로1가 중앙로1가
4,중앙로2가,5111010500,강원특별자치도 춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,NaN,4.436733e+04,북한C,4.436733e+04,100.0,100.0,1,춘천시 중앙로2가 중앙로2가
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,장산리,5183035038,강원특별자치도 양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,NaN,2.518901e+06,100.0,100.0,1,양양군 강현면 장산리
1363,전진리,5183035039,강원특별자치도 양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,NaN,9.178029e+05,100.0,100.0,1,양양군 강현면 전진리
1364,주청리,5183035040,강원특별자치도 양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,NaN,4.658669e+05,100.0,100.0,1,양양군 강현면 주청리
1365,적은리,5183035041,강원특별자치도 양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,NaN,1.361706e+06,100.0,100.0,1,양양군 강현면 적은리


In [ ]:
# 점유율 자료 읽기 및 전처리 
점유율 = pd.read_excel("E:/Coding/TMDL/전국오염원조사/단위유역별 점유율.xlsx")

# 동리코드, 단위유역, 시군 그룹으로 해서 생활계부터 유원지 열까지 각각 합계 계산
점유율 = 점유율.groupby(["동리코드", "단위유역", "시군"]).agg(
    
    
    
    
점유율

,동리코드,단위유역,소유역,시도,시군,읍면동,동리,생활계,축산계,산업계,...,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,면적(㎢),면적점유율(%)
0,평창군 대관령면 유천리,골지A,골지A01,강원도,평창군,대관령면,유천리,0.000000,0.000000,0.0,...,0.558518,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0422,0.001298
1,평창군 대관령면 병내리,골지A,골지A01,강원도,평창군,대관령면,병내리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0452,0.001184
2,평창군 대관령면 차항리,골지A,골지A01,강원도,평창군,대관령면,차항리,99.446379,99.446379,0.0,...,98.260804,100.000000,0.0,100.0,99.054376,100.0,100.000000,0.0,25.7825,0.991963
3,평창군 대관령면 횡계리,골지A,골지A01,강원도,평창군,대관령면,횡계리,100.000000,100.000000,100.0,...,100.000000,100.000000,100.0,100.0,100.000000,100.0,100.000000,100.0,61.8261,0.999173
4,평창군 대관령면 수하리,골지A,골지A01,강원도,평창군,대관령면,수하리,48.231325,48.231325,0.0,...,43.786371,33.222964,0.0,100.0,27.344476,0.0,42.631997,0.0,4.1141,0.129039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,영월군 수주면 무릉리,주천A,주천A11,강원도,영월군,수주면,무릉리,81.727353,81.727353,100.0,...,96.940144,83.169701,100.0,100.0,75.651966,0.0,100.000000,0.0,0.0000,1.000000
2409,영월군 수주면 법흥리,주천A,주천A11,강원도,영월군,수주면,법흥리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.000000
2410,영월군 수주면 도원리,주천A,주천A11,강원도,영월군,수주면,도원리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.000000
2411,영월군 수주면 무릉리,평창A,평창A09,강원도,영월군,수주면,무릉리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.000000


In [35]:
# 데이터 정리
# 필요한 열 선택 및 변수명 지정
축산계_정리 = 축산계_원본.iloc[:, [0, 1, 11, 12, 13, 14, 15, 16, 17]].copy()
축산계_정리.columns = ["연도", "농장명", "시군", "읍면동", "리", "본번", "부번", "축종", "사육두수"]

# 사육두수와 연도 컬럼을 숫자로 변환; 사육두수의 결측치는 0으로 채움
for col in ["사육두수", "연도"]:
    축산계_정리[col] = pd.to_numeric(축산계_정리[col], errors='coerce')
축산계_정리["사육두수"] = 축산계_정리["사육두수"].fillna(0)
# 사육두수가 0인 행 제거
축산계_정리 = 축산계_정리[축산계_정리["사육두수"] != 0]

# 축종 변환: 조건에 따른 축종명 매핑
def convert_축종(축종):
    if 축종 == "한우(소)":
        return "한우"
    elif 축종 == "유우(젖소)":
        return "젖소"
    elif 축종 == "돼지":
        return "돼지"
    elif 축종 == "마필(말)":
        return "말"
    elif 축종 in ["산양(염소포함)", "면양(육양포함)", "사슴"]:
        return "양, 사슴"
    elif 축종 == "개":
        return "개"
    elif 축종 in ["닭", "오리", "타조", "가금기타"]:
        return "가금"
    else:
        return "-"

축산계_정리["축종"] = 축산계_정리["축종"].apply(convert_축종)

축산계_정리

,연도,농장명,시군,읍면동,리,본번,부번,축종,사육두수
0,2023,성기춘,춘천시,온의동,NaN,432,NaN,한우,20
1,2023,이재호,춘천시,삼천동,NaN,142,2,한우,80
4,2023,춘천승마장,춘천시,송암동,NaN,157,NaN,말,25
5,2023,김창경,춘천시,신동,NaN,1206,12,한우,10
6,2023,김희원,춘천시,신동,NaN,637,8,한우,20
...,...,...,...,...,...,...,...,...,...
11081,2023,이지환,양양군,강현면,장산리,784,2,한우,86
11082,2023,털보농장,양양군,강현면,장산리,139,NaN,한우,33
11083,2023,홍성훈,양양군,강현면,장산리,705,NaN,한우,42
11084,2023,강산농장,양양군,강현면,장산리,100,2,한우,30


In [36]:
# 주소 정리
# 함수: 본번 문자열 정리 (괄호, 쉼표, "외" 관련 문자 제거)
def clean_bonbeon(x):
    if pd.isnull(x):
        return x
    x = str(x)
    # 괄호 안의 내용 제거
    x = re.sub(r"\(.+", "", x)
    # 쉼표와 그 이후 내용 제거
    x = re.sub(r",.*", "", x)
    # "외"로 시작하는 부분 제거
    x = re.sub(r"(외).*", "", x)
    return x.strip()

def clean_bubun(x):
    if pd.isnull(x):
        return x
    x = str(x)
    x = re.sub(r"(외).*", "", x)
    return x.strip()

축산계_정리["본번"] = 축산계_정리["본번"].apply(clean_bonbeon)
축산계_정리["부번"] = 축산계_정리["부번"].apply(clean_bubun)

# 법정동명 생성: 시군, 읍면동, 그리고 리가 결측이면 읍면동 사용, 합친 후에 좌우 여백 제거
축산계_정리["법정동명"] = 축산계_정리.apply(
    lambda row: f"{"강원특별자치도"} {row['시군']} {row['읍면동']} {row['리'] if pd.notnull(row['리']) and row['리'] != '' else ''}",
    axis=1
)

# 법정동명 좌우 여백 제거
축산계_정리["법정동명"] = 축산계_정리["법정동명"].str.strip()

# 주소 합치기 
def make_address(row):
    법정동명 = str(row["법정동명"])
    본번 = str(row["본번"])
    부번 = str(row["부번"]) if (row["부번"] not in [0, "", np.nan]) else ""
    
    # 부번이 존재하면 '-'와 함께 붙임
    부번_part = "" if 부번=="" else f"-{부번}"
    return f"{법정동명} {본번}{부번_part}".strip()

축산계_정리["주소"] = 축산계_정리.apply(make_address, axis=1)

축산계_정리

,연도,농장명,시군,읍면동,리,본번,부번,축종,사육두수,법정동명,주소
0,2023,성기춘,춘천시,온의동,NaN,432,NaN,한우,20,강원특별자치도 춘천시 온의동,강원특별자치도 춘천시 온의동 432
1,2023,이재호,춘천시,삼천동,NaN,142,2,한우,80,강원특별자치도 춘천시 삼천동,강원특별자치도 춘천시 삼천동 142-2
4,2023,춘천승마장,춘천시,송암동,NaN,157,NaN,말,25,강원특별자치도 춘천시 송암동,강원특별자치도 춘천시 송암동 157
5,2023,김창경,춘천시,신동,NaN,1206,12,한우,10,강원특별자치도 춘천시 신동,강원특별자치도 춘천시 신동 1206-12
6,2023,김희원,춘천시,신동,NaN,637,8,한우,20,강원특별자치도 춘천시 신동,강원특별자치도 춘천시 신동 637-8
...,...,...,...,...,...,...,...,...,...,...,...
11081,2023,이지환,양양군,강현면,장산리,784,2,한우,86,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 784-2
11082,2023,털보농장,양양군,강현면,장산리,139,NaN,한우,33,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 139
11083,2023,홍성훈,양양군,강현면,장산리,705,NaN,한우,42,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 705
11084,2023,강산농장,양양군,강현면,장산리,100,2,한우,30,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 100-2


In [37]:
# 단위유역 좌측 조인 (법정동명 기준)
축산계_정리 = 축산계_정리.merge(단위유역, on="법정동명", how="left")

축산계_정리

,연도,농장명,시군,읍면동,리,본번,부번,축종,사육두수,법정동명,주소,단위유역
0,2023,성기춘,춘천시,온의동,NaN,432,NaN,한우,20,강원특별자치도 춘천시 온의동,강원특별자치도 춘천시 온의동 432,북한C
1,2023,이재호,춘천시,삼천동,NaN,142,2,한우,80,강원특별자치도 춘천시 삼천동,강원특별자치도 춘천시 삼천동 142-2,북한C
2,2023,춘천승마장,춘천시,송암동,NaN,157,NaN,말,25,강원특별자치도 춘천시 송암동,강원특별자치도 춘천시 송암동 157,북한C
3,2023,김창경,춘천시,신동,NaN,1206,12,한우,10,강원특별자치도 춘천시 신동,강원특별자치도 춘천시 신동 1206-12,북한C
4,2023,김희원,춘천시,신동,NaN,637,8,한우,20,강원특별자치도 춘천시 신동,강원특별자치도 춘천시 신동 637-8,북한C
...,...,...,...,...,...,...,...,...,...,...,...,...
9818,2023,이지환,양양군,강현면,장산리,784,2,한우,86,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 784-2,NaN
9819,2023,털보농장,양양군,강현면,장산리,139,NaN,한우,33,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 139,NaN
9820,2023,홍성훈,양양군,강현면,장산리,705,NaN,한우,42,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 705,NaN
9821,2023,강산농장,양양군,강현면,장산리,100,2,한우,30,강원특별자치도 양양군 강현면 장산리,강원특별자치도 양양군 강현면 장산리 100-2,NaN
